In [1]:
import re
from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_dataframe(df):
    # 1. 저자와 책 제목을 title 컬럼으로 만들어줌 (띄어쓰기로 구분)
    df = df[df['TITLE_NM'].str.contains("[\uac00-\ud7a3\u3131-\u3163\uac01-\ud7a6]+")] # 한국어 외 삭제
    df['TITLE_NM'] = df['TITLE_NM'].apply(lambda x: re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', x)) # 숫자 외 특수문자 제거

    # 2. TITLE_NM의 양쪽 공백 제거
    df['TITLE_NM'] = df['TITLE_NM'].str.strip()

    # 3. 중복 값 삭제
    df = df.drop_duplicates(subset='TITLE_NM')

    return df

In [3]:
df = pd.read_csv("DataSet_row/BOOK_HIST.csv", encoding='utf8')
df

,TITLE_NM,AUTHR_NM,KDC_NM,COUNTING,LBRRY_CD
0,# i 세대,진 트웬지 지음 ;김현정 옮김,331.233,6,5300
1,# 법률가,김동일 ;어윤경 ;최윤정 지음,372.68,8,5301
2,#1 to infinity,머라이어 캐리 노래,673.53,1,4717
3,#16 Shop Talk,Juwanda G. Ford,82.0,2,30007
4,#2 JUNIE B. JONES AND A LITTLE MONKEY BUSINESS,by Barbara Park ;illustrated by Danise Brunkus,843.0,6,4502
...,...,...,...,...,...
2641346,��킨 이야기,다니자키 준이치로 [지음],833.6,2,4600
2641347,��킨 이야기,지은이: 다니자키 준이치로 ;옮긴이: 박연정,833.6,6,4717
2641348,����이,글·그림: 이우,813.8,4,4714
2641349,����이,이우,813.8,1,25902


In [4]:
len(df)

2641351

In [12]:
(list(df['TITLE_NM']))

['# i 세대',
 '# 법률가',
 '#1 to infinity',
 '#16 Shop Talk',
 '#2 JUNIE B. JONES AND A LITTLE MONKEY BUSINESS',
 '#CEO',
 '#Girlboss',
 '#Goldilocks :(a)hashtag cautionary tale',
 '#Goldilocks =  : A Hashtag Cautionary Tale',
 '#Now:the surprising truth about the power of now',
 '#SF #페미니즘 #그녀들의이야기',
 '#i세대 =  : 스마트폰을 손에 쥐고 자란 요즘 세대 이야기',
 '#i세대:스마트폰을 손에 쥐고 자란 요즘 세대 이야기',
 '#걸보스',
 '#걸보스 : 훔친 책을 팔던 소녀, 5년 만에 1000억대 CEO가 되다',
 '#걸보스 :훔친 책을 팔던 소녀, 5년 만에 1000억대 CEO가 되다',
 '#걸보스 =  : 훔친 책을 팔던 소녀, 5년 만에 1000억대 CEO가 되다',
 '#걸보스:훔친 책을 팔던 소녀, 5년 만에 1000억대 CEO가 되다',
 '#걸보스:훔친 책을 팔던 소녀, 5년 만에 1000억대 CEO가 되다',
 '#교육자',
 '#구멍',
 '#구멍',
 '#구멍',
 '#구멍  =',
 '#구멍 =',
 '#너에게',
 '#너에게',
 '#너에게 :',
 '#너에게 :Special Edition :너에게 보내는 편지_완글',
 '#너에게 :너에게 보내는 편지, 완글',
 '#너에게 :너에게 보내는 편지, 완글',
 '#너에게 :너에게 보내는 편지, 완글 :Special Edition',
 '#너에게 :너에게 보내는 편지, 완글 :Special Edition',
 '#너에게 =  : 너에게 보내는 편지, 완글',
 '#너에게 스페셜 에디션',
 '#너에게:',
 '#너에게:special edition',
 '#너에게:너에게 보내는 편지, 완글',
 '#너에게:너에게 보내는 편지, 완글',
 '#너에게:너에

In [6]:
data = preprocess_dataframe(df)
data = data.head(10000)
data

,TITLE_NM,AUTHR_NM,KDC_NM,COUNTING,LBRRY_CD
0,세대,진 트웬지 지음 ;김현정 옮김,331.233,6,5300
1,법률가,김동일 ;어윤경 ;최윤정 지음,372.68,8,5301
10,페미니즘 그녀들의이야기,김효진 지음,809.9,2,8400
11,세대 스마트폰을 손에 쥐고 자란 요즘 세대 이야기,진 트웬지 지음,331.233,1,4504
12,세대스마트폰을 손에 쥐고 자란 요즘 세대 이야기,진 트웬지 지음;김현정 옮김,331.233,6,30002
...,...,...,...,...,...
11622,10대가 알아야 할 프로그래밍과 코딩이야기 10대를 위한 최고의 프로그래밍코딩...,우혁,5.133,1,17001
11623,10대가 알아야 할 프로그래밍과 코딩이야기10대를 위한 최고의 프로그래밍코딩 입문서,"우혁,이설아 [공]지음",5.133,2,8420
11624,10대가 알아야 할미래 부의 이동,신지나;김재남;민준홍 [공]지음,321.97,1,10510
11626,10대가 알아야 할미래 부의 이동 와 인공지능이 만드는 4차 산업혁명 시대 진...,"신지나, 김재남;, 민준홍 [공]지음",327.04,10,6009


In [7]:
len(data)

10000

In [15]:
data.columns

Index(['TITLE_NM', 'AUTHR_NM', 'KDC_NM', 'COUNTING', 'LBRRY_CD'], dtype='object')

In [16]:
list(data['TITLE_NM'])

['세대',
 '법률가',
 '페미니즘 그녀들의이야기',
 '세대    스마트폰을 손에 쥐고 자란 요즘 세대 이야기',
 '세대스마트폰을 손에 쥐고 자란 요즘 세대 이야기',
 '걸보스',
 '걸보스  훔친 책을 팔던 소녀 5년 만에 1000억대 가 되다',
 '걸보스 훔친 책을 팔던 소녀 5년 만에 1000억대 가 되다',
 '걸보스    훔친 책을 팔던 소녀 5년 만에 1000억대 가 되다',
 '걸보스훔친 책을 팔던 소녀 5년 만에 1000억대 가 되다',
 '교육자',
 '구멍',
 '너에게',
 '너에게   너에게 보내는 편지완글',
 '너에게 너에게 보내는 편지 완글',
 '너에게    너에게 보내는 편지 완글',
 '너에게 스페셜 에디션',
 '너에게너에게 보내는 편지 완글',
 '너에게너에게 보내는 편지 완글스페셜 에디션',
 '너에게스페셜 에디션',
 '만화가',
 '문학은위험하다지금 여기의 페미니즘과 독자 시대의 한국문학',
 '미투 그리고 나와 너',
 '살아있다',
 '살아있다 비디오녹화자료',
 '생태소설',
 '생태시',
 '성소수자',
 '소셜 쓰고 앉았네',
 '소셜쓰고앉았네',
 '소셜쓰고앉았네 9년차 마케터가 말하는 지극히 객관적이지만 극도로 주관적인 소셜미디어 실전서',
 '스크래치 코딩 알고리듬',
 '스크래치 코딩 알고리듬   수알치와 함께하는 프로그래밍 예제',
 '씬  정지민 장편소설',
 '씬정지민 장편소설',
 '여자 공학인 4차 산업혁명',
 '예능작가',
 '예능작가  예능작가 16인의 생생한 방송 이야기',
 '예능작가 예능작가 16인의 생생한 방송 이야기',
 '예능작가예능작가 16인의 생생한 방송 이야기',
 '월화수목육아일',
 '월화수목육아일 육아 퇴근을 꿈꾸는 엄마들을 위한 힐링북',
 '월화수목육아일    육아 퇴근을 꿈꾸는 엄마들을 위한 힐링북',
 '의학자',
 '인스타 종이데코',
 '인포그래픽 성경',
 '일상의 파괴울고웃고춤추며 사랑하리 쿠바처럼',
 '점장아님주의 편의점의 4년 5개월 편의

In [225]:
def analyze_similarity(data, new_title, kdc,n=10):
    # 저자 + 책제목 / 군집화
    #data["corpus"] = (pd.Series(data[["AUTHR_NM", "TITLE_NM"]].fillna('').values.tolist()).str.join(" "))
    data["KDC_NM"] = data["KDC_NM"].astype("str")
    data = data[data["KDC_NM"].str[0]==kdc]
    data = data.sort_values("COUNTING", ascending=False)
    
    data = list(data["title"])
    
    hannanum = Hannanum()
    tokens = [hannanum.morphs(datum) for datum in data]


    vectorizer = CountVectorizer()
    vectorized_data = vectorizer.fit_transform([' '.join(token) for token in tokens])


    new_title_tokenized = hannanum.morphs(new_title)
    new_title_vectorized = vectorizer.transform([' '.join(new_title_tokenized)])


    similarities = cosine_similarity(vectorized_data, new_title_vectorized)
    similar_indices = similarities.flatten().argsort()[-n:][::-1]

    similar_titles = [data[index] for index in similar_indices]

    return similar_titles

In [226]:
analyze_similarity(data, new_title, kdc="8", n=10)

['오은 지음 마음의 일 ',
 '이선 지음 개를 낳았다 ',
 '오은 지음 마음의 일',
 '안상현 지음 달의 마음 내 마음이 네 마음을',
 '임진아 외 지음 일하는 마음과 앓는 마음',
 '임진아천현우하완김예지김준박문수 등 지음 일하는 마음과 앓는 마음',
 '임진아 일하는 마음과 앓는 마음 ',
 '임진아 외 지음 일하는 마음과 앓는 마음',
 '임진아 외 지음 일하는 마음과 앓는 마음 ',
 '고두현 지음 시를 놓고 살았다 사랑을 놓고 살았다 ']

In [40]:
def analyze_similarity_list(data, new_title_list, kdc_list, n=10): # 유사도를 바탕으로 상위 10개
    data["KDC_NM"] = data["KDC_NM"].astype("str")
    
    data_1 = data[data["KDC_NM"].str[0] == "1"]
    data_1 = data_1.sort_values("COUNTING", ascending=False)
    data_2 = data[data["KDC_NM"].str[0] == "2"]
    data_2 = data_2.sort_values("COUNTING", ascending=False)
    data_3 = data[data["KDC_NM"].str[0] == "3"]
    data_3 = data_3.sort_values("COUNTING", ascending=False)
    data_4 = data[data["KDC_NM"].str[0] == "4"]
    data_4 = data_4.sort_values("COUNTING", ascending=False)
    data_5 = data[data["KDC_NM"].str[0] == "5"]
    data_5 = data_5.sort_values("COUNTING", ascending=False)
    data_6 = data[data["KDC_NM"].str[0] == "6"]
    data_6 = data_6.sort_values("COUNTING", ascending=False)
    data_7 = data[data["KDC_NM"].str[0] == "7"]
    data_7 = data_7.sort_values("COUNTING", ascending=False)
    data_8 = data[data["KDC_NM"].str[0] == "8"]
    data_8 = data_8.sort_values("COUNTING", ascending=False)
    data_9 = data[data["KDC_NM"].str[0] == "9"]
    data_9 = data_9.sort_values("COUNTING", ascending=False)
    
    hannanum = Hannanum()
    
    tokens1 = [hannanum.morphs(datum) for datum in data_1["TITLE_NM"]]
    tokens2 = [hannanum.morphs(datum) for datum in data_2["TITLE_NM"]]
    tokens3 = [hannanum.morphs(datum) for datum in data_3["TITLE_NM"]]
    tokens4 = [hannanum.morphs(datum) for datum in data_4["TITLE_NM"]]
    tokens5 = [hannanum.morphs(datum) for datum in data_5["TITLE_NM"]]
    tokens6 = [hannanum.morphs(datum) for datum in data_6["TITLE_NM"]]
    tokens7 = [hannanum.morphs(datum) for datum in data_7["TITLE_NM"]]
    tokens8 = [hannanum.morphs(datum) for datum in data_8["TITLE_NM"]]
    tokens9 = [hannanum.morphs(datum) for datum in data_9["TITLE_NM"]]

    vectorizer1 = CountVectorizer()
    vectorized_data1 = vectorizer1.fit_transform([' '.join(token) for token in tokens1])
    
    vectorizer2 = CountVectorizer()
    vectorized_data2 = vectorizer2.fit_transform([' '.join(token) for token in tokens2])
    
    vectorizer3 = CountVectorizer()
    vectorized_data3 = vectorizer3.fit_transform([' '.join(token) for token in tokens3])
    
    vectorizer4 = CountVectorizer()
    vectorized_data4 = vectorizer4.fit_transform([' '.join(token) for token in tokens4])
    
    vectorizer5 = CountVectorizer()
    vectorized_data5 = vectorizer5.fit_transform([' '.join(token) for token in tokens5])
    
    vectorizer6 = CountVectorizer()
    vectorized_data6 = vectorizer6.fit_transform([' '.join(token) for token in tokens6])
    
    vectorizer7 = CountVectorizer()
    vectorized_data7 = vectorizer7.fit_transform([' '.join(token) for token in tokens7])
    
    vectorizer8 = CountVectorizer()
    vectorized_data8 = vectorizer8.fit_transform([' '.join(token) for token in tokens8])
    
    vectorizer9 = CountVectorizer()
    vectorized_data9 = vectorizer9.fit_transform([' '.join(token) for token in tokens9])
    
    similar_titles_list = []
    for kdc, new_title_list in zip(kdc_list, new_title_list):
        if kdc == "1":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer1.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data1, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_1.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "2":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer2.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data2, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_2.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "3":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer3.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data3, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_3.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "4":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer4.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data4, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_4.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "5":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer5.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data5, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_5.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "6":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer6.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data6, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_6.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "7":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer7.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data7, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_7.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "8":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer8.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data8, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_8.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)
        elif kdc == "9":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer9.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data9, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_titles = [data_9.iloc[index]["TITLE_NM"] for index in similar_indices]
            similar_titles_list.append(similar_titles)

    return similar_titles_list


In [51]:
def analyze_similarity_list(data, new_title_list, kdc_list, n=20): #유사도 상위 2개에서 카운팅을 바탕으로 상위 10개
    data["KDC_NM"] = data["KDC_NM"].astype("str")

    data_1 = data[data["KDC_NM"].str[0] == "1"]
    data_1 = data_1.sort_values("COUNTING", ascending=False)
    data_2 = data[data["KDC_NM"].str[0] == "2"]
    data_2 = data_2.sort_values("COUNTING", ascending=False)
    data_3 = data[data["KDC_NM"].str[0] == "3"]
    data_3 = data_3.sort_values("COUNTING", ascending=False)
    data_4 = data[data["KDC_NM"].str[0] == "4"]
    data_4 = data_4.sort_values("COUNTING", ascending=False)
    data_5 = data[data["KDC_NM"].str[0] == "5"]
    data_5 = data_5.sort_values("COUNTING", ascending=False)
    data_6 = data[data["KDC_NM"].str[0] == "6"]
    data_6 = data_6.sort_values("COUNTING", ascending=False)
    data_7 = data[data["KDC_NM"].str[0] == "7"]
    data_7 = data_7.sort_values("COUNTING", ascending=False)
    data_8 = data[data["KDC_NM"].str[0] == "8"]
    data_8 = data_8.sort_values("COUNTING", ascending=False)
    data_9 = data[data["KDC_NM"].str[0] == "9"]
    data_9 = data_9.sort_values("COUNTING", ascending=False)

    hannanum = Hannanum()

    tokens1 = [hannanum.morphs(datum) for datum in data_1["TITLE_NM"]]
    tokens2 = [hannanum.morphs(datum) for datum in data_2["TITLE_NM"]]
    tokens3 = [hannanum.morphs(datum) for datum in data_3["TITLE_NM"]]
    tokens4 = [hannanum.morphs(datum) for datum in data_4["TITLE_NM"]]
    tokens5 = [hannanum.morphs(datum) for datum in data_5["TITLE_NM"]]
    tokens6 = [hannanum.morphs(datum) for datum in data_6["TITLE_NM"]]
    tokens7 = [hannanum.morphs(datum) for datum in data_7["TITLE_NM"]]
    tokens8 = [hannanum.morphs(datum) for datum in data_8["TITLE_NM"]]
    tokens9 = [hannanum.morphs(datum) for datum in data_9["TITLE_NM"]]

    vectorizer1 = CountVectorizer()
    vectorized_data1 = vectorizer1.fit_transform([' '.join(token) for token in tokens1])

    vectorizer2 = CountVectorizer()
    vectorized_data2 = vectorizer2.fit_transform([' '.join(token) for token in tokens2])

    vectorizer3 = CountVectorizer()
    vectorized_data3 = vectorizer3.fit_transform([' '.join(token) for token in tokens3])

    vectorizer4 = CountVectorizer()
    vectorized_data4 = vectorizer4.fit_transform([' '.join(token) for token in tokens4])

    vectorizer5 = CountVectorizer()
    vectorized_data5 = vectorizer5.fit_transform([' '.join(token) for token in tokens5])

    vectorizer6 = CountVectorizer()
    vectorized_data6 = vectorizer6.fit_transform([' '.join(token) for token in tokens6])

    vectorizer7 = CountVectorizer()
    vectorized_data7 = vectorizer7.fit_transform([' '.join(token) for token in tokens7])

    vectorizer8 = CountVectorizer()
    vectorized_data8 = vectorizer8.fit_transform([' '.join(token) for token in tokens8])

    vectorizer9 = CountVectorizer()
    vectorized_data9 = vectorizer9.fit_transform([' '.join(token) for token in tokens9])

    similar_titles_list = []
    for kdc, new_title_list in zip(kdc_list, new_title_list):
        if kdc == "1":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer1.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data1, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_1.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "2":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer2.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data2, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_2.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "3":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer3.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data3, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_3.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "4":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer4.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data4, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_4.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "5":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer5.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data5, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_5.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "6":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer6.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data6, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_6.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "7":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer7.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data7, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_7.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "8":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer8.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data8, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_8.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)
        elif kdc == "9":
            new_title = new_title_list
            new_title_tokenized = hannanum.morphs(new_title)
            new_title_vectorized = vectorizer9.transform([' '.join(new_title_tokenized)]).toarray()

            similarities = cosine_similarity(vectorized_data9, new_title_vectorized)
            similar_indices = similarities.flatten().argsort()[-n:][::-1]

            similar_books = data_9.iloc[similar_indices]
            similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
            similar_titles = similar_books["TITLE_NM"].tolist()
            similar_titles_list.append(similar_titles)

    return similar_titles_list

In [50]:
new_title  = ["인생의 진리는 무엇인가", "기도를 드리는 마음", "통계가 제일 좋아", "수학이 너무 재미있어", "정밀한 기계의 부품", "조각을 위한 손기술", "발음을 위한 기술", "내가 당신에 닿는 순간", "서유럽의 역사"]
new_kdc = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [37]:
df['KDC_NM'] = df['KDC_NM'].astype('str')
df['KDC_NM'].str[0]

0          3
1          3
2          6
3          8
4          8
          ..
2641346    8
2641347    8
2641348    8
2641349    8
2641350    8
Name: KDC_NM, Length: 2641351, dtype: object

In [52]:
%time
result = analyze_similarity_list(data, new_title, new_kdc)
result

CPU times: total: 0 ns
Wall time: 0 ns
[319851  24956 314040 139232 156786 319852 319849 319850 187149 319841
 254979 153658  85108 319827  18705 156751 319826  17941  12479 100273]
                             TITLE_NM  AUTHR_NM KDC_NM  COUNTING  LBRRY_CD
1201583                         당신의 빛  강경수 글·그림  813.8        84     30005
1201925                         당신의 신     김숨 지음  813.7        63     30007
1203839                     당신이 좋아진 날    송정연 지음    818        61      8305
1734657                 순간의 나와 영원의 당신    손현녕 지음    818        47      8300
2128351                          이 순간     능행 지음    818        11     28200
2085748                        울지마 당신  이용현 글·사진    818         9     26000
2346543  책등에 베이다    당신과 내가 책을 꺼내드는 순간     이로 지음    818         5      6000
1201348                     당신의 모든 순간        강풀    818         4     30100
1201849                         당신의 손    한상숙 지음  813.6         4     28200
2338079                       참 좋은 당신    김용택 지음  811.6         4    

[['내 삶의 의미는 무엇인가',
  '나란 무엇인가',
  '무엇이 탁월한 삶인가',
  '좋은 삶이란 무엇인가',
  '지성사란 무엇인가',
  '정서란 무엇인가',
  '쿤달리니란 무엇인가',
  '무엇이 좋은 삶인가',
  '윤리란 무엇인가',
  '요가란 무엇인가'],
 ['밤에 드리는 기도',
  '마음에서 우러나오는 기도',
  '주님의 마음에 이르는 기도',
  '마음 탓이다',
  '살아나는 기도',
  '마음의 길',
  '마음 비움',
  '중보자의 마음을 품는 기도',
  '천 개의 기도',
  '기도하는 삶'],
 ['통계의 마술 재미있는 통계 이야기',
  '내 소리가 제일 커',
  '통계의 거짓말',
  '내가 제일 잘한 일',
  '통계로 본 대한민국',
  '통계의 함정조작된 통계 불량통계의 위험과 부작용',
  '난 숨바꼭질이 제일 좋아',
  '통계의 함정조작된 통계 불량 통계의 위험과 부작용',
  '표본조사론 통계',
  '제일 멋진 나'],
 ['수학 유령의 미스터리마술수학    쉽고 재미있는 수학 개념원리',
  '재미있는 수학 이야기',
  '수학 유령의미스터리 캠핑 수학    쉽고 재미있는 수학 개념원리',
  '만화보다 재미있는 3학년 수학 연습장 수학 교과서가 재미있는 이야기로 변신',
  '만화보다 재미있는 수학 연습장    수학 교과서가 재미있는 이야기로 변신',
  '수학이 재미있어지는 수학사',
  '수학 유령의 미스터리 요리 수학 쉽고 재미있는 수학 개념원리',
  '수학 유령의 미스터리 캠핑 수학 쉽고 재미있는 수학 개념원리',
  '수학 유령의 미스터리 마술 수학 쉽고 재미있는 수학 개념원리',
  '수학 유령의 미스터리 캠핑 수학  쉽고 재미있는 수학 개념원리'],
 ['비눗방울 기계',
  '요리조리 열어보는 기계의 원리',
  '요리조리 열어 보는 기계의 원리',
  '자이언트어마어마한 탈것과 기계',
  '공사감독 핸드북 기계',
  '꿈꾸는 기계의 진화',
  '뚝딱뚝딱 도구와 기계',
 

In [53]:
for raw_book, re_books in zip(new_title, result):
    print(raw_book, ":" , re_books)
    print("")

인생의 진리는 무엇인가 : ['내 삶의 의미는 무엇인가', '나란 무엇인가', '무엇이 탁월한 삶인가', '좋은 삶이란 무엇인가', '지성사란 무엇인가', '정서란 무엇인가', '쿤달리니란 무엇인가', '무엇이 좋은 삶인가', '윤리란 무엇인가', '요가란 무엇인가']

기도를 드리는 마음 : ['밤에 드리는 기도', '마음에서 우러나오는 기도', '주님의 마음에 이르는 기도', '마음 탓이다', '살아나는 기도', '마음의 길', '마음 비움', '중보자의 마음을 품는 기도', '천 개의 기도', '기도하는 삶']

통계가 제일 좋아 : ['통계의 마술 재미있는 통계 이야기', '내 소리가 제일 커', '통계의 거짓말', '내가 제일 잘한 일', '통계로 본 대한민국', '통계의 함정조작된 통계 불량통계의 위험과 부작용', '난 숨바꼭질이 제일 좋아', '통계의 함정조작된 통계 불량 통계의 위험과 부작용', '표본조사론 통계', '제일 멋진 나']

수학이 너무 재미있어 : ['수학 유령의 미스터리마술수학    쉽고 재미있는 수학 개념원리', '재미있는 수학 이야기', '수학 유령의미스터리 캠핑 수학    쉽고 재미있는 수학 개념원리', '만화보다 재미있는 3학년 수학 연습장 수학 교과서가 재미있는 이야기로 변신', '만화보다 재미있는 수학 연습장    수학 교과서가 재미있는 이야기로 변신', '수학이 재미있어지는 수학사', '수학 유령의 미스터리 요리 수학 쉽고 재미있는 수학 개념원리', '수학 유령의 미스터리 캠핑 수학 쉽고 재미있는 수학 개념원리', '수학 유령의 미스터리 마술 수학 쉽고 재미있는 수학 개념원리', '수학 유령의 미스터리 캠핑 수학  쉽고 재미있는 수학 개념원리']

정밀한 기계의 부품 : ['비눗방울 기계', '요리조리 열어보는 기계의 원리', '요리조리 열어 보는 기계의 원리', '자이언트어마어마한 탈것과 기계', '공사감독 핸드북 기계', '꿈꾸는 기계의 진화', '뚝딱뚝딱 도구와 기계', '무료동영상이 있는 소방설비기사 실기 기계

In [8]:
df = pd.read_csv("./test_data/TEST_YANGCHEON.csv")
df

,TITLE_NM,AUTHR_NM,KDC_NM,LBRRY_NM,LOAN_DATE
0,미키7 : 애드워드 애슈턴 SF 장편소설,애드워드 애슈턴 지음 ; 배지혜 옮김,843.6-애56ㅁ,[스마트]양천중앙,22/12/31 21:45:38
1,"마흔, 부부가 함께 은퇴합니다 : 5년 만에 40대 조기 은퇴에 성공한, 금융맹 부...",김다현 지음,327.04-김22ㅁ,[스마트]양천25시(오목교역),22/12/31 21:15:56
2,(조셉 필라테스의)필라테스 바이블,"조셉 필라테스,저드 로빈스,린 반 휴트-로빈스 [공]엮음 ; 원정희 옮김",517.32-필292ㅍ,[스마트] 신정네거리,22/12/31 20:51:24
3,헤어질 결심 각본,"정서경,박찬욱 지음",812.66-정54ㅎ,[스마트]양천25시(오목교역),22/12/31 17:36:14
4,너무 잘하려고 애쓰지 마라,나태주 지음,811.6-나883너,[스마트]양천중앙,22/12/31 17:35:47
...,...,...,...,...,...
9951,"지금, 인생의 체력을 길러야 할 때 : 나를 인생 1순위에 놓기 위해 꼭 필요한 1...","제니퍼 애슈턴 지음, 김지혜 옮김",517.3-애56ㅈ=2,[스마트]놀터(목3동목동누리어린이공원),22/01/01 15:07:43
9952,"즐거워, 빵과 커피가 있으면 : 매일 가고 싶은 빵집·카페 78곳",이하성 글과 사진,594.019-이92ㅈ,[스마트]양천25시(오목교역),22/01/01 13:54:12
9953,부서진 여름 : 이정명 장편소설,이정명 지음,813.6-이74부=3,[스마트]양천25시(오목교역),22/01/01 13:52:41
9954,(곽재식의)미래를 파는 상점 : SF 소설가가 그리는 미래과학 세상,곽재식 지음,504-곽73ㅁ,[스마트]양천중앙,22/01/01 13:20:03


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9956 entries, 0 to 9955
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   TITLE_NM   9956 non-null   object
 1   AUTHR_NM   9956 non-null   object
 2   KDC_NM     9956 non-null   object
 3   LBRRY_NM   9956 non-null   object
 4   LOAN_DATE  9956 non-null   object
dtypes: object(5)
memory usage: 389.0+ KB


In [52]:
df['LOAN_DATE'] = pd.to_datetime(df['LOAN_DATE'])
df['LOAN_DATE'] = df['LOAN_DATE'].dt.strftime('%m')
df['LOAN_DATE'] = df['LOAN_DATE'].astype('int')
df.head()

,TITLE_NM,AUTHR_NM,KDC_NM,LBRRY_NM,LOAN_DATE
0,미키7 : 애드워드 애슈턴 SF 장편소설,애드워드 애슈턴 지음 ; 배지혜 옮김,843.6-애56ㅁ,[스마트]양천중앙,12
1,"마흔, 부부가 함께 은퇴합니다 : 5년 만에 40대 조기 은퇴에 성공한, 금융맹 부...",김다현 지음,327.04-김22ㅁ,[스마트]양천25시(오목교역),12
2,(조셉 필라테스의)필라테스 바이블,"조셉 필라테스,저드 로빈스,린 반 휴트-로빈스 [공]엮음 ; 원정희 옮김",517.32-필292ㅍ,[스마트] 신정네거리,12
3,헤어질 결심 각본,"정서경,박찬욱 지음",812.66-정54ㅎ,[스마트]양천25시(오목교역),12
4,너무 잘하려고 애쓰지 마라,나태주 지음,811.6-나883너,[스마트]양천중앙,12


In [53]:
df['LOAN_DATE'].unique()

array([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1])

In [67]:
df[df['LOAN_DATE']==6]

,TITLE_NM,AUTHR_NM,KDC_NM,LBRRY_NM,LOAN_DATE


In [11]:
def calculate_similarity(a_list, b_list):
    max_similarity_list = []
    
    hannanum = Hannanum()
    a_tokens = [hannanum.morphs(title) for title in a_list]
    vectorizer = CountVectorizer()
    a_vectorized = vectorizer.fit_transform([' '.join(tokens) for tokens in a_tokens])


    for b_title in b_list:

        b_tokens = hannanum.morphs(b_title)


        b_vectorized = vectorizer.transform([' '.join(b_tokens)])
    

        similarity = cosine_similarity(a_vectorized, b_vectorized).max()
        max_similarity_list.append(similarity)

    average_similarity = sum(max_similarity_list) / len(max_similarity_list)
    return average_similarity

def maek_score(data, test_data, n = 20):
    data["KDC_NM"] = data["KDC_NM"].astype("str")

    test_data['LOAN_DATE'] = pd.to_datetime(test_data['LOAN_DATE'])
    test_data['LOAN_DATE'] = test_data['LOAN_DATE'].dt.strftime('%m')
    test_data['LOAN_DATE'] = test_data['LOAN_DATE'].astype('int')
    test_data['TITLE_NM'] = test_data['TITLE_NM'].apply(lambda x: re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', x))
    test_data['TITLE_NM'] = test_data['TITLE_NM'].str.strip()
    
    data_1 = data[data["KDC_NM"].str[0] == "1"]
    data_1 = data_1.sort_values("COUNTING", ascending=False)
    data_2 = data[data["KDC_NM"].str[0] == "2"]
    data_2 = data_2.sort_values("COUNTING", ascending=False)
    data_3 = data[data["KDC_NM"].str[0] == "3"]
    data_3 = data_3.sort_values("COUNTING", ascending=False)
    data_4 = data[data["KDC_NM"].str[0] == "4"]
    data_4 = data_4.sort_values("COUNTING", ascending=False)
    data_5 = data[data["KDC_NM"].str[0] == "5"]
    data_5 = data_5.sort_values("COUNTING", ascending=False)
    data_6 = data[data["KDC_NM"].str[0] == "6"]
    data_6 = data_6.sort_values("COUNTING", ascending=False)
    data_7 = data[data["KDC_NM"].str[0] == "7"]
    data_7 = data_7.sort_values("COUNTING", ascending=False)
    data_8 = data[data["KDC_NM"].str[0] == "8"]
    data_8 = data_8.sort_values("COUNTING", ascending=False)
    data_9 = data[data["KDC_NM"].str[0] == "9"]
    data_9 = data_9.sort_values("COUNTING", ascending=False)
    
    hannanum = Hannanum()
    
    tokens1 = [hannanum.morphs(datum) for datum in data_1["TITLE_NM"]]
    tokens2 = [hannanum.morphs(datum) for datum in data_2["TITLE_NM"]]
    tokens3 = [hannanum.morphs(datum) for datum in data_3["TITLE_NM"]]
    tokens4 = [hannanum.morphs(datum) for datum in data_4["TITLE_NM"]]
    tokens5 = [hannanum.morphs(datum) for datum in data_5["TITLE_NM"]]
    tokens6 = [hannanum.morphs(datum) for datum in data_6["TITLE_NM"]]
    tokens7 = [hannanum.morphs(datum) for datum in data_7["TITLE_NM"]]
    tokens8 = [hannanum.morphs(datum) for datum in data_8["TITLE_NM"]]
    tokens9 = [hannanum.morphs(datum) for datum in data_9["TITLE_NM"]]
    
    vectorizer1 = CountVectorizer()
    vectorized_data1 = vectorizer1.fit_transform([' '.join(token) for token in tokens1])
    
    vectorizer2 = CountVectorizer()
    vectorized_data2 = vectorizer2.fit_transform([' '.join(token) for token in tokens2])
    
    vectorizer3 = CountVectorizer()
    vectorized_data3 = vectorizer3.fit_transform([' '.join(token) for token in tokens3])
    
    vectorizer4 = CountVectorizer()
    vectorized_data4 = vectorizer4.fit_transform([' '.join(token) for token in tokens4])
    
    vectorizer5 = CountVectorizer()
    vectorized_data5 = vectorizer5.fit_transform([' '.join(token) for token in tokens5])
    
    vectorizer6 = CountVectorizer()
    vectorized_data6 = vectorizer6.fit_transform([' '.join(token) for token in tokens6])
    
    vectorizer7 = CountVectorizer()
    vectorized_data7 = vectorizer7.fit_transform([' '.join(token) for token in tokens7])
    
    vectorizer8 = CountVectorizer()
    vectorized_data8 = vectorizer8.fit_transform([' '.join(token) for token in tokens8])
    
    vectorizer9 = CountVectorizer()
    vectorized_data9 = vectorizer9.fit_transform([' '.join(token) for token in tokens9])
    
    score_list = []
    for num in range(1, 12):
        print(f"{num}월")
        similar_titles_list = []
        month_data = test_data[test_data['LOAN_DATE']==num]
        new_title_list = list(month_data['TITLE_NM'])
        kdc_list = list(month_data['KDC_NM'].str[0])
        for kdc, new_title in zip(kdc_list, new_title_list):
            if kdc == "1":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer1.transform([' '.join(new_title_tokenized)]).toarray()
        
                similarities = cosine_similarity(vectorized_data1, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
        
                similar_books = data_1.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "2":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer2.transform([' '.join(new_title_tokenized)]).toarray()
        
                similarities = cosine_similarity(vectorized_data2, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
        
                similar_books = data_2.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "3":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer3.transform([' '.join(new_title_tokenized)]).toarray()
        
                similarities = cosine_similarity(vectorized_data3, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
        
                similar_books = data_3.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "4":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer4.transform([' '.join(new_title_tokenized)]).toarray()
    
                similarities = cosine_similarity(vectorized_data4, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
    
                similar_books = data_4.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "5":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer5.transform([' '.join(new_title_tokenized)]).toarray()
    
                similarities = cosine_similarity(vectorized_data5, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
    
                similar_books = data_5.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "6":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer6.transform([' '.join(new_title_tokenized)]).toarray()
    
                similarities = cosine_similarity(vectorized_data6, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
    
                similar_books = data_6.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "7":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer7.transform([' '.join(new_title_tokenized)]).toarray()
    
                similarities = cosine_similarity(vectorized_data7, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
    
                similar_books = data_7.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "8":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer8.transform([' '.join(new_title_tokenized)]).toarray()

                similarities = cosine_similarity(vectorized_data8, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
    
                similar_books = data_8.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(10)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            elif kdc == "9":
                new_title = new_title
                new_title_tokenized = hannanum.morphs(new_title)
                new_title_vectorized = vectorizer9.transform([' '.join(new_title_tokenized)]).toarray()
        
                similarities = cosine_similarity(vectorized_data9, new_title_vectorized)
                similar_indices = similarities.flatten().argsort()[-n:][::-1]
    
                similar_books = data_9.iloc[similar_indices]
                similar_books = similar_books.sort_values("COUNTING", ascending=False).head(5)
                similar_titles = similar_books["TITLE_NM"].tolist()
                similar_titles_list.append(similar_titles)
            else:
                pass
        after_book = list(test_data[test_data['LOAN_DATE']==num+1]['TITLE_NM'])
        similar_titles_list = [item for sublist in similar_titles_list for item in sublist]
        result = calculate_similarity(after_book, similar_titles_list)
        print(result)
        score_list.append(result)
    
    return score_list

In [66]:
df['LOAN_DATE']

0       1
1       1
2       1
3       1
4       1
       ..
9951    1
9952    1
9953    1
9954    1
9955    1
Name: LOAN_DATE, Length: 9956, dtype: int32

In [12]:
score_result = maek_score(data, test_data=df, n=20)

1월
0.4724871043837118
2월
0.47782049869078125
3월


KeyboardInterrupt: 

In [70]:
print(1/0)

ZeroDivisionError: division by zero